#### 2019-12-17
# Current/Bias Concatenation Testing Notebook


This is the notebook primarily made for testing that the newly added concatenation methods in splopter are working as expected.


In [2]:
%matplotlib tk
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import xarray as xr
import pandas as pd
import scipy.io as sio
import sys
import re
import os
import glob
import copy
import pathlib as pth
import importlib
sys.path.append('/home/jleland/Coding/Projects/flopter')
import flopter.spice.splopter as spl
import flopter.spice.tdata as td
import flopter.spice.inputparser as inp
import flopter.core.ivdata as iv
import flopter.core.fitters as fts
import flopter.core.fitdata as fd
import flopter.core.lputils as lpu
import flopter.core.constants as c

## Paths and Matfile IO
This section deals with file io and selecting the right .mat files. This needs to be run for the latter sections to work.

In [3]:
# lowdens_dir = pth.Path('/home/jleland/Spice/spice2/bin/data_local_m2/lowdens_anglescan')
lowdens_dir = pth.Path('/home/jleland/data/external/spice/')
os.chdir(lowdens_dir)

In [4]:
lps = lpu.MagnumProbes()

flush_probe = copy.deepcopy(lps.probe_l)
flush_probe.theta_p = 0.0
flush_probe.d_perp = 0.0

angled_probe = copy.deepcopy(lps.probe_l)
angled_probe.d_perp = 0.0
print(angled_probe.theta_p)

0.17453292519943295


In [5]:
skippable_scans = {
    'marconi/lowdens_anglescan/inpartest_as',
    'marconi/lowdens_anglescan/angled_tiltscan_ext',
    'marconi/lowdens_anglescan/sprobe_tiltscan',
    'marconi/lowdens_anglescan/recessed_tiltscan',
    'marconi/lowdens_anglescan/rearwall_tiltscan',
}
single_sims = {
    'marconi/lowdens_anglescan/angled_1',
}

In [6]:
# Assemble all folder names for the lowdens anglescans

non_standard_variables = {'ProbePot', 'npartproc', 'Nz', 'Nzmax', 'Ny', 'count', 'Npc', 'snumber', 'nproc'}
desired_variables = td.DEFAULT_REDUCED_DATASET | non_standard_variables

# # UNCOMMENT IF YOU WANT THIS

# scans_searchstr = '*/lowdens_anglescan/*'
# # angles_search_str = '/*[!.{yml, inp}]/backup*'
# angles_search_str = '/*[!.{yml, inp}]'

# all_run_dirs = {}
# scans = glob.glob(scans_searchstr)
# scans = set(scans) - skippable_scans
# for scan in scans:
#     if scan in single_sims:
#         print(f'Single sim {scan} found')
#         all_run_dirs[scan] = [scan]
#     else:
#         all_run_dirs[scan] = glob.glob(scan + angles_search_str)
    
#     print(f'{scan}: {all_run_dirs[scan]}\n')


# Individual looks at simulations and backup reconstruction

In [18]:
sim_path = 'marconi/lowdens_anglescan/recessed_tiltscan_redo/alpha_yz_-1.0'
sim_path_single = 'marconi/lowdens_anglescan/recessed_tiltscan_redo/alpha_yz_-1.0'

# sim_path = 'marconi/lowdens_anglescan/angled_tiltscan/alpha_yz_-4.0'
# sim_path_single = 'marconi/lowdens_anglescan/angled_tiltscan/alpha_yz_-4.0'
# sim_path = 'marconi/lowdens_anglescan/angled_tiltscan/alpha_yz_-8.0'
# sim_path_single = 'marconi/lowdens_anglescan/angled_tiltscan/alpha_yz_-8.0'
# sim_path = 'marconi/lowdens_anglescan/rearwall_tiltscan_redo.1/alpha_yz_-10.0'
# sim_path_single = 'marconi/lowdens_anglescan/rearwall_tiltscan_redo.1/alpha_yz_-10.0'

# sim_path_single = 'marconi/restart_test/sprobe_control_2-13-1_'
# sim_path = 'marconi/restart_test/sprobe_20hr_multirestart_2-13-1_4c'
# sim_path_single = 'marconi/restart_test/sprobe_9hr_multirestart'
# sim_path = 'marconi/restart_test/sprobe_9hr_multirestart'


try:
    print('\n\n --- Making concatted splopter ---')
    splopter = spl.Splopter(lowdens_dir / sim_path, reduce=desired_variables)
    splopter.prepare(denormaliser_fl=True, homogenise_fl=True, find_se_temp_fl=False)
except spl.FailedSimulationException as e:
    print(f'FAILED: {e}')

print('\n\n --- Making single splopter ---')
splopter_single = spl.Splopter(lowdens_dir / sim_path_single, reduce=desired_variables)
splopter_single.prepare(denormaliser_fl=True, homogenise_fl=True, find_se_temp_fl=False)



 --- Making concatted splopter ---
Looking for a suitable backup to use instead.
Useable backup found at /home/jleland/data/external/spice/marconi/lowdens_anglescan/recessed_tiltscan_redo/alpha_yz_-1.0/backup_20191126-1416/t-alpha_yz_-1.0.mat
Loading backup backup_20191126-1416 (1 of 5) for current and bias concatenation
Loading backup backup_20191127-1127 (2 of 5) for current and bias concatenation
Stripping head for backup 2 of 5
Loading backup backup_20191128-1528 (3 of 5) for current and bias concatenation
Stripping head for backup 3 of 5
Loading backup backup_20191128-1944 (4 of 5) for current and bias concatenation
Detected finish, stripping tail for backup 4 of 5
Stripping head for backup 4 of 5
Found an odd number of spike transitions


 --- Making single splopter ---
Looking for a suitable backup to use instead.
Useable backup found at /home/jleland/data/external/spice/marconi/lowdens_anglescan/recessed_tiltscan_redo/alpha_yz_-1.0/backup_20191126-1416/t-alpha_yz_-1.0.mat
Loa

In [8]:
sim_path_pth = pth.Path(sim_path)
log_file = list(sim_path_pth.glob('log.ongoing.out'))[0]
# print(subprocess.check_output(['grep', "'remaining'", str(log_file)]))

def get_h_remaining_lines(log_file):
    time_left_lines = []
    with open(str(log_file), 'r') as f:
        for line in f.readlines():
            if re.search(' % done', line):
                time_left_lines.append(line.strip())
    return time_left_lines

final_percentage = int(get_h_remaining_lines(log_file)[-1].split()[0]) / 100
print(final_percentage)

0.9


In [10]:
print(splopter.parser.has_sufficient_sweep(final_percentage, voltage_threshold=5))


False


In [10]:
fig, axes = plt.subplots(3, 3, sharex='row', sharey='row')
axes[0][0].plot(splopter.iv_data['V'])
axes[1][0].plot(splopter.iv_data['I'])
axes[2][0].plot(splopter.iv_data['V'], splopter.iv_data['I'])

axes[0][1].plot(splopter.raw_data['V'])
axes[1][1].plot(splopter.raw_data['I'])
axes[2][1].plot(splopter.raw_data['V'], splopter.raw_data['I'])

axes[0][2].plot(splopter_single.raw_data['V'])
axes[1][2].plot(splopter_single.raw_data['I'])
axes[2][2].plot(splopter_single.raw_data['V'][:-2], splopter_single.raw_data['I'])


ValueError: x and y must have same first dimension, but have shapes (3271,) and (3273,)

In [9]:
print(np.min(splopter.raw_data['V']))
print(np.min(splopter_single.raw_data['V']))

-0.25862496890402714
-9.950000000000001


In [19]:
biases = {}
currents = {}
for bu_folder in splopter.backup_folders:
    bu_tfile_path, _ = splopter.get_ta_filenames(bu_folder)
    bu_tdata = td.Spice2TData(bu_folder / bu_tfile_path, variable_names=[td.T,
                                                                         c.DIAG_PROBE_POT,
                                                                         td.OBJECTSCURRENTE,
                                                                         td.OBJECTSCURRENTI,
                                                                         td.OBJECTSENUM,
                                                                         td.OBJECTS])
    bu_time, bu_probe_bias, bu_probe_current_e, bu_probe_current_i = splopter.get_tdata_raw_iv(bu_tdata)
    bu_probe_current_tot = bu_probe_current_e + bu_probe_current_i
    biases[bu_folder.name] = bu_probe_bias
    currents[bu_folder.name] = bu_probe_current_tot
 

In [20]:
# Plot all current/voltage traces in backups

# fig, axes = plt.subplots(2, 4, sharey='row')
prev_current = 0.0
for i, bu_folder in enumerate(list(biases.keys())):
    print(bu_folder)
    
    fig, axes = plt.subplots(2, 1, sharex='row', sharey='row')
    axes[0].plot(biases[bu_folder], label=f'{bu_folder}')
    axes[1].plot(currents[bu_folder], label=f'{bu_folder}')
    axes[0].legend()
    axes[1].legend()
#     axes[1].set_ylim(-150, 50)
    axes[1].axhline(prev_current, **c.AX_LINE_DEFAULTS)
    
    current_ind = np.arange(len(currents[bu_folder]))
    prev_current_arr = np.zeros_like(current_ind) + prev_current
    axes[1].fill_between(current_ind, 0.9 * prev_current_arr, 1.1 * prev_current_arr, facecolor='yellow', alpha=0.5)
    
    prev_current = currents[bu_folder][-1]

if splopter.iv_data is not None:
    fig, axes = plt.subplots(2, 2, sharex='col', sharey='row')
    axes[0][0].plot(splopter.raw_data['V'])
    axes[1][0].plot(splopter.raw_data['I'])
    axes[0][1].plot(splopter.iv_data['V'])
    axes[1][1].plot(splopter.iv_data['I'])


backup_20191126-1416
backup_20191127-1127
backup_20191128-1528
backup_20191128-1944
backup_20191128-2001


In [12]:
# Plot first 2 current/voltage traces in backups

fig, axes = plt.subplots(2, 4, sharey='row')
prev_current = 0.0
for i, bu_folder in enumerate(list(biases.keys())[:2]):
    print(bu_folder)
    
#     fig, axes = plt.subplots(2, 1, sharex='row', sharey='row')
    axes[0][i].plot(biases[bu_folder], label=f'{bu_folder}')
    axes[1][i].plot(currents[bu_folder], label=f'{bu_folder}')
    axes[0][i].legend()
    axes[1][i].legend()
    axes[1][i].set_ylim(-150, 50)
    axes[1][i].axhline(prev_current, **c.AX_LINE_DEFAULTS)
    
    current_ind = np.arange(len(currents[bu_folder]))
    prev_current_arr = np.zeros_like(current_ind) + prev_current
    axes[1][i].fill_between(current_ind, 0.9 * prev_current_arr, 1.1 * prev_current_arr, facecolor='yellow', alpha=0.5)
    
    prev_current = currents[bu_folder][-1]

if splopter.iv_data is not None:
#     fig, axes = plt.subplots(2, 2, sharex='col', sharey='row')
    axes[0][2].plot(splopter.raw_data['V'])
    axes[1][2].plot(splopter.raw_data['I'])
    axes[0][3].plot(splopter.iv_data['V'])
    axes[1][3].plot(splopter.iv_data['I'])
    
axes[0][0].set_title('First 24 Hours')
axes[0][1].set_title('Second 24 Hours')
axes[0][2].set_title('Stitched traces')
axes[0][3].set_title('Extracted IV Arrays')

axes[0][0].set_ylabel('Probe Potential (from Diagnostic)')
axes[1][0].set_ylabel('Objectscurrent total')

backup_20190809-0021
backup_20190809-0029


Text(0, 0.5, 'Objectscurrent total')

/home/jleland/.pyenv/versions/3.6.9/lib/python3.6/tkinter/__init__.py:749: UserWarning: Creating legend with loc="best" can be slow with large amounts of data.
  func(*args)
/home/jleland/.pyenv/versions/3.6.9/lib/python3.6/tkinter/__init__.py:1705: UserWarning: Creating legend with loc="best" can be slow with large amounts of data.
  return self.func(*args)


In [23]:
# Test bed for stripping/pruning functions

backup_of_interest = list(biases.keys())[0]
current = currents[backup_of_interest]
voltage = biases[backup_of_interest]
prev_current = 4.001464046845688

fig, axes = plt.subplots(2, 1, sharex=True, sharey=True)

axes[0].plot(current, label=f'{bu_folder}')
axes[0].axhline(prev_current, **c.AX_LINE_DEFAULTS)

print(round(voltage[-1], 2))

if round(voltage[-1], 2) >= 10.00:

    big_spikes = np.where(np.abs(current) > 1000)
    if len(big_spikes[0]) == 0:
        pass
    if len(big_spikes[0]) >= 1:
        end_spike = big_spikes[0][np.argmax(np.abs(current[big_spikes]))]
        print(big_spikes, end_spike)
        axes[1].plot(np.arange(end_spike), current[:end_spike], label=f'pre-spike')
        axes[1].plot(np.arange(end_spike+1, len(current)), current[end_spike+1:], label=f'post-spike')

        axes[1].legend()
#     axes[0].set_ylim(-300, 50)

do_current_concat = True
indiced_oi = None
new_current, start_point = strip_head(current, prev_current)

axes[1].plot(np.arange(start_point, len(current)), new_current)
for i in range(2):
    axes[i].axvline(start_point, **c.AX_LINE_DEFAULTS)

-9.9
prev_current = 4.001464046845688
Successfully used primary method


In [34]:
def strip_head(current, prev_current):
    print(f'prev_current = {prev_current}')
    
    # Primary method - find first point roughly equal to previous end value
    start_points = np.where(np.logical_and(current >= prev_current * 0.8, current <= prev_current * 1.2))
    if len(start_points[0]) > 0:
        for start_point in start_points[0]:
            if not any(current[start_point:] == 0.0):
                print('Successfully used primary method')
                return current[start_point:], start_point
            else:
                print(f'Failed with start_point {start_point}')
    
    # Secondary method - find last zero in current array and take the succeeding value
    zero_indices = np.where(current == 0.0)[0]
    if current[-1] != 0.0 and len(zero_indices) > 0:
        # Take start point to be the value one place along from the final 0
        start_point = np.max(zero_indices + 1)
        print('Successfully used secondary method')
        return current[start_point:], start_point
    
    return None, None 

def strip_tail(current, threshold=10000):
    # Strip tail section if present
    big_spikes = np.where(np.abs(current) > threshold)
    if len(big_spikes[0]) == 0:
        print('No spikes found on tail.')
    elif len(big_spikes[0]) >= 1:
        end_spike = big_spikes[0][np.argmax(np.abs(current[big_spikes]))]
        current = current[:end_spike]
    return current

def group_consecutives(vals, step=1):
    """Return list of consecutive lists of numbers from vals (number list)."""
    run = []
    result = [run]
    expect = None
    for v in vals:
        if (v == expect) or (expect is None):
            run.append(v)
        else:
            run = [v]
            result.append(run)
        expect = v + step
    return result


# def group_spikes(voltage, tolerance=3):
#     # Determine location of start and end points of the spikes by locating values >3 sigma from mean difference
#     bias_diff = np.diff(voltage)
#     diff_mean = np.mean(bias_diff)
#     diff_spread = tolerance * np.std(bias_diff)
#     diff_outliers = np.where(np.logical_or(bias_diff > diff_mean + diff_spread, bias_diff < diff_mean - diff_spread))[0]
    
#     if len(diff_outliers) % 2 != 0:
#         print('Not an even number of spike transitions')
        
#         sp_start = 0
#         sp_end = 1
#         pairs = []
#         while sp_end < len(diff_outliers): 
#             print(f'sp_start = {sp_start}, sp_end = {sp_end}')
#             if np.sign(bias_diff[diff_outliers][sp_start]) != np.sign(bias_diff[diff_outliers][sp_end]):
#                 pairs.append([diff_outliers[sp_start], diff_outliers[sp_end]])
#                 sp_start = sp_end + 1
#                 sp_end = sp_start + 1
#             else:
#                 sp_end += 1
#         return pairs 
#     else:
#         return list(zip(diff_outliers[::2], diff_outliers[1::2]))
    
def group_spikes(voltage, tolerance=5):
    # Determine location of start and end points of the spikes by locating values >3 sigma from mean difference
    bias_diff = np.diff(voltage)
#     diff_mean = np.mean(bias_diff)
#     diff_spread = tolerance * np.std(bias_diff)
#     diff_outliers = \
#         np.where(np.logical_or(bias_diff > diff_mean + diff_spread, bias_diff < diff_mean - diff_spread))[0]
    diff_outliers = np.where(np.abs(bias_diff) >= 0.06)[0]

    if len(diff_outliers) % 2 != 0:
        print('Found an odd number of spike transitions')

        start_spike = 0
        end_spike = 1
        pairs = []
        # Group by diffs where there is a change of sign
        while end_spike < len(diff_outliers):
            if np.sign(bias_diff[diff_outliers][start_spike]) != np.sign(bias_diff[diff_outliers][end_spike]):
                pairs.append([diff_outliers[start_spike], diff_outliers[end_spike]])
                start_spike = end_spike + 1
                end_spike = start_spike + 1
            else:
                end_spike += 1
        return pairs
    else:
        return list(zip(diff_outliers[::2], diff_outliers[1::2]))

def prune_voltage(voltage):
    # Find consectuive problem values and group them
    edges = group_spikes(voltage)
    print(edges)
    
    # Construct a replacement array to replace each spike
    if len(edges) > 0:
        for start_spike, end_spike in edges:
            if start_spike == 0 or end_spike == len(voltage) - 1:
                raise NotImplementedError('Edge cases not accounted for yet')

            start_v = voltage[start_spike]
            end_v = voltage[end_spike+1]
            length = end_spike - start_spike
            replacement = np.linspace(start_v, end_v, length + 2)

            voltage[start_spike:end_spike+2] = replacement

    return voltage

In [22]:
concatted_current = np.array([])
concatted_voltage = np.array([])
prev_current = 0.0
prev_voltage = 0.0

for i, backup_of_interest in enumerate(biases.keys()):
    
    current = currents[backup_of_interest].copy()
    voltage = biases[backup_of_interest].copy()
    finished_fl = False
    
    print(current[0], current[-1])
    
    if i == 0:
        # If first step then whole current array is left for concatting and final 
        # current value is stored for next iteration to assist in stripping
        prev_current = current[-1]
    
    fig, ax = plt.subplots()
    ax.plot(np.arange(len(current)), current, label='unaltered')
    
    if round(voltage[-1], 2) >= 9.95 or i+1 == len(biases):
        # If final step then strip off the end spike, zeroes and appended IV, if 
        # present
        print(f'Detected finish, stripping tail for backup {i}')
        finished_fl = True
        current = strip_tail(current)
        ax.plot(np.arange(len(current)), current, label='strip_end')
    
    if i >= 1:
        # If not the first step then strip the leading zeroes/spikes
        print(f'Stripping head for backup {i}')
        post_tailstrip_length = len(current)
        current, start_point = strip_head(current, prev_current)
        prev_current = current[-1]
    
        print(start_point)
        ax.plot(np.arange(start_point, post_tailstrip_length), current, label='strip_beginning')
    
    concatted_current = np.append(concatted_current, current)
    ax.legend()
    
    if finished_fl:
        fig, axes = plt.subplots(2, 1)
        axes[0].plot(concatted_current)
        axes[1].plot(voltage, color='r')
        
        test_voltage = voltage.copy()
        # Final version of voltage is stored in completed run, needs some minor pruning
        concatted_voltage = prune_voltage(voltage)
        break
      

fig, axes = plt.subplots(2, 1)
axes[0].plot(concatted_current)
axes[1].plot(concatted_voltage, color='r')

           
    

0.0 3.904295812460917
-0.01562925015633248 4.001464046845688
Stripping head for backup 1
prev_current = 3.904295812460917
Failed with start_point 20
Successfully used primary method
996
0.0 -34.80987879784542
Stripping head for backup 2
prev_current = 4.001464046845688
Successfully used primary method
910
0.0 -39.46977733674909
Detected finish, stripping tail for backup 3
No spikes found on tail.
Stripping head for backup 3
prev_current = -34.80987879784542
Successfully used secondary method
1301
Found an odd number of spike transitions
[[175072, 175073], [1863072, 1863074], [1865929, 1870593], [1875257, 3464072], [3464075, 3465682]]


In [27]:
print(len(splopter.iv_data['V']), len(splopter.iv_data['I']), len(splopter.iv_data['t']))

spikes = np.where(np.abs(np.diff(splopter.iv_data['V'])) < 5)
print(f'length of spikes {len(spikes[0])}')
iv_data_backup = splopter.iv_data['V']
# splopter.iv_data['V'] = splopter.iv_data['V'][spikes]

plt.plot(splopter.iv_data['t'], splopter.iv_data['V'])
plt.show()


533 533 533
length of spikes 532


In [15]:
dummy_voltage = np.append(np.zeros(10000)-9.95, np.linspace(-10.00,10.00,10000))
dummy_voltage[5123:5128] = 0.0
dummy_voltage[15093:15125] = -9.95

plt.figure()
plt.plot(dummy_voltage)

In [23]:
plt.figure()
plt.plot(test_voltage.copy())

The below is the current method for removing spikes from the concatted bias data. It seems that there's a length mismatch between the bias and current arrays, which is causing some headaches. In a specific test example removing the spikes seems to prevent this, but this doesn't seem to apply generally as a non-restarted (and therefore non-concattable) run is seeing the same issue. Will need to check the homogenisation method to check what it's doing. 

In [35]:
splopt = splopter
# voltage = splopt.raw_data['V'].copy()
voltage = test_voltage.copy()
# voltage = dummy_voltage.copy()

# Inital rought method for removing the spikes in voltage data. 
print(len(splopt.iv_data['V']), len(splopt.iv_data['I']), len(splopt.iv_data['t']))
voltage_backup = voltage.copy()

bias_diff = np.diff(voltage)
diff_mean = np.mean(bias_diff)
diff_spread = 5 * np.std(bias_diff)
bias_outliers = np.where(np.logical_or(bias_diff > diff_mean + diff_spread, bias_diff < diff_mean - diff_spread))
bias_outliers = np.where(np.abs(bias_diff) > 0.1)

fig, axes = plt.subplots(2, sharex=True)
axes[0].plot(bias_diff)
axes[0].axhline(diff_mean, color='red', linestyle='dashed', linewidth=.8)
axes[0].axhspan(diff_mean + diff_spread, diff_mean - diff_spread, color='red', alpha=0.25)
axes[0].plot(bias_outliers[0], bias_diff[bias_outliers], 'x')

edges = group_spikes(voltage)

print('\nPrinting with unpacking...')
for beg, end in edges:
    print(f'beg:{beg}, end:{end}')
    
for start_spike, end_spike in edges:
    if start_spike == 0 or end_spike == len(voltage) - 1:
        raise NotImplementedError('Edge cases not accounted for yet')
        
    print(start_spike, end_spike)
    start_v = voltage[start_spike]
    end_v = voltage[end_spike+1]
    length = end_spike - start_spike
    replacement = np.linspace(start_v, end_v, length + 2)
    print(f'start: {start_v}, end: {end_v}, length: {length + 2}')
    print(replacement)
    voltage_backup[start_spike:end_spike+2] = replacement

fig, ax = plt.subplots(2, sharex=True, sharey=True)
ax[0].plot(voltage, 'x-')
ax[1].plot(voltage_backup, 'x-')

# splopter.iv_data['V'] = splopter.iv_data['V'][spikes]

# plt.plot(splopter_single.iv_data['t'], splopter_single.iv_data['V'])
plt.show()

1085 1085 1085
Found an odd number of spike transitions

Printing with unpacking...
beg:175072, end:175073
beg:1863072, end:1863074
beg:3464072, end:3465682
175072 175073
start: -9.9, end: -9.9, length: 3
[-9.9 -9.9 -9.9]
1863072 1863074
start: -9.9, end: -9.9, length: 4
[-9.9 -9.9 -9.9 -9.9]
3464072 3465682
start: 7.150000000000001, end: 7.200000000000001, length: 1612
[7.15       7.15003104 7.15006207 ... 7.19993793 7.19996896 7.2       ]


In [25]:
edges = group_spikes(dummy_voltage)

def group_spikes(voltage, tolerance=5):
    bias_diff = np.diff(voltage)
    diff_mean = np.mean(bias_diff)
    diff_spread = tolerance * np.std(bias_diff)
    diff_outliers = np.where(np.logical_or(bias_diff > diff_mean + diff_spread, bias_diff < diff_mean - diff_spread))
    
    if len(diff_outliers[0]) % 2 != 0:
        raise NotImplementedError('Not an even number of spike transitions')
    
    return list(zip(diff_outliers[0][::2], diff_outliers[0][1::2]))

NameError: name 'dummy_voltage' is not defined

In [22]:
for i in zip([1,2,3], [1,3,5]):
    print(i)

(1, 1)
(2, 3)
(3, 5)
